#### 2019-12-17
# Current/Bias Concatenation Testing Notebook


This is the notebook primarily made for testing that the newly added concatenation methods in splopter are working as expected.


In [1]:
%matplotlib tk
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import xarray as xr
import pandas as pd
import scipy.io as sio
import sys
import os
import glob
import copy
import pathlib as pth
import importlib
sys.path.append('/home/jleland/Coding/Projects/flopter')
import flopter.spice.splopter as spl
import flopter.spice.tdata as td
import flopter.spice.homogenise as hmg
import flopter.core.ivdata as iv
import flopter.core.fitters as fts
import flopter.core.fitdata as fd
import flopter.core.lputils as lpu
import flopter.core.constants as c

## Paths and Matfile IO
This section deals with file io and selecting the right .mat files. This needs to be run for the latter sections to work.

In [2]:
# lowdens_dir = pth.Path('/home/jleland/Spice/spice2/bin/data_local_m2/lowdens_anglescan')
lowdens_dir = pth.Path('/home/jleland/data/external/spice/')
os.chdir(lowdens_dir)

In [3]:
lps = lpu.MagnumProbes()

flush_probe = copy.deepcopy(lps.probe_l)
flush_probe.theta_p = 0.0
flush_probe.d_perp = 0.0

angled_probe = copy.deepcopy(lps.probe_l)
angled_probe.d_perp = 0.0
print(angled_probe.theta_p)

0.17453292519943295


In [4]:
skippable_scans = {
    'marconi/lowdens_anglescan/inpartest_as',
    'marconi/lowdens_anglescan/angled_tiltscan_ext',
    'marconi/lowdens_anglescan/sprobe_tiltscan',
    'marconi/lowdens_anglescan/recessed_tiltscan',
    'marconi/lowdens_anglescan/rearwall_tiltscan',
}
single_sims = {
    'marconi/lowdens_anglescan/angled_1',
}

In [5]:
# Assemble all folder names for the lowdens anglescans

non_standard_variables = {'ProbePot', 'npartproc', 'Nz', 'Nzmax', 'Ny', 'count', 'Npc', 'snumber', 'nproc'}
desired_variables = td.DEFAULT_REDUCED_DATASET | non_standard_variables

# # UNCOMMENT IF YOU WANT THIS

# scans_searchstr = '*/lowdens_anglescan/*'
# # angles_search_str = '/*[!.{yml, inp}]/backup*'
# angles_search_str = '/*[!.{yml, inp}]'

# all_run_dirs = {}
# scans = glob.glob(scans_searchstr)
# scans = set(scans) - skippable_scans
# for scan in scans:
#     if scan in single_sims:
#         print(f'Single sim {scan} found')
#         all_run_dirs[scan] = [scan]
#     else:
#         all_run_dirs[scan] = glob.glob(scan + angles_search_str)
    
#     print(f'{scan}: {all_run_dirs[scan]}\n')


# Individual looks at simulations and backup reconstruction

In [6]:
sim_path = 'marconi/lowdens_anglescan/recessed_tiltscan_redo/alpha_yz_-2.0'
sim_path_single = 'marconi/lowdens_anglescan/recessed_tiltscan_redo/alpha_yz_-2.0'
# sim_path_single = 'marconi/restart_test/sprobe_control_2-13-1_'
# sim_path = 'marconi/restart_test/sprobe_20hr_multirestart_2-13-1_4c'

print('\n\n --- Making concatted splopter ---')
splopter = spl.Splopter(lowdens_dir / sim_path, reduce=desired_variables)
splopter.prepare(denormaliser_fl=True, homogenise_fl=True, find_se_temp_fl=False)

print('\n\n --- Making single splopter ---')
splopter_single = spl.Splopter(lowdens_dir / sim_path_single, reduce=desired_variables)
splopter_single.prepare(denormaliser_fl=True, homogenise_fl=True, find_se_temp_fl=False, backup_concat_fl=False)



 --- Making concatted splopter ---
Looking for a suitable backup to use instead.
Useable backup found at /home/jleland/data/external/spice/marconi/lowdens_anglescan/recessed_tiltscan_redo/alpha_yz_-2.0/backup_20191126-1416/t-alpha_yz_-2.0.mat
Loading backup /home/jleland/data/external/spice/marconi/lowdens_anglescan/recessed_tiltscan_redo/alpha_yz_-2.0/backup_20191126-1416 (1 of 5) for current and bias concatenation
Loading backup /home/jleland/data/external/spice/marconi/lowdens_anglescan/recessed_tiltscan_redo/alpha_yz_-2.0/backup_20191127-0320 (2 of 5) for current and bias concatenation
Loading backup /home/jleland/data/external/spice/marconi/lowdens_anglescan/recessed_tiltscan_redo/alpha_yz_-2.0/backup_20191127-0335 (3 of 5) for current and bias concatenation
Loading backup /home/jleland/data/external/spice/marconi/lowdens_anglescan/recessed_tiltscan_redo/alpha_yz_-2.0/backup_20191127-0349 (4 of 5) for current and bias concatenation
Loading backup /home/jleland/data/external/spic

/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/numpy/core/_methods.py:217: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/numpy/core/_methods.py:186: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/numpy/core/_methods.py:209: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [8]:
fig, axes = plt.subplots(3, 3, sharex='row', sharey='row')
axes[0][0].plot(splopter.iv_data['V'])
axes[1][0].plot(splopter.iv_data['I'])
axes[2][0].plot(splopter.iv_data['V'], splopter.iv_data['I'])

axes[0][1].plot(splopter.raw_data['V'])
axes[1][1].plot(splopter.raw_data['I'])
axes[2][1].plot(splopter.raw_data['V'], splopter.raw_data['I'])

axes[0][2].plot(splopter_single.raw_data['V'])
axes[1][2].plot(splopter_single.raw_data['I'])
axes[2][2].plot(splopter_single.raw_data['V'][:-2], splopter_single.raw_data['I'])


In [7]:
biases = {}
currents = {}
for bu_folder in splopter.backup_folders:
    bu_tfile_path, _ = splopter.get_ta_filenames(bu_folder)
    bu_tdata = td.Spice2TData(bu_folder / bu_tfile_path, variable_names=[td.T,
                                                                         c.DIAG_PROBE_POT,
                                                                         td.OBJECTSCURRENTE,
                                                                         td.OBJECTSCURRENTI,
                                                                         td.OBJECTSENUM,
                                                                         td.OBJECTS])
    bu_time, bu_probe_bias, bu_probe_current_e, bu_probe_current_i = splopter.get_tdata_raw_iv(bu_tdata)
    bu_probe_current_tot = bu_probe_current_e + bu_probe_current_i
    biases[bu_folder.name] = bu_probe_bias
    currents[bu_folder.name] = bu_probe_current_tot
 

In [8]:
# Plot all current/voltage traces in backups

# fig, axes = plt.subplots(3, 1, sharex='row', sharey='row')
prev_current = 0.0
for bu_folder in list(biases.keys())[:]:
    fig, axes = plt.subplots(2, 1, sharex='row', sharey='row')
    axes[0].plot(biases[bu_folder][::1000], label=f'{bu_folder}')
    axes[1].plot(currents[bu_folder], label=f'{bu_folder}')
    axes[0].legend()
    axes[1].legend()
#     axes[1].set_ylim(-250, 50)
    axes[1].axhline(prev_current, **c.AX_LINE_DEFAULTS)
    
    current_ind = np.arange(len(currents[bu_folder]))
    prev_current_arr = np.zeros_like(current_ind) + prev_current
    axes[1].fill_between(current_ind, 0.9 * prev_current_arr, 1.1 * prev_current_arr, facecolor='yellow', alpha=0.5)
    
    prev_current = currents[bu_folder][-1]


fig, axes = plt.subplots(2, 2, sharex='col', sharey='row')
axes[0][0].plot(splopter.iv_data['V'])
axes[1][0].plot(splopter.iv_data['I'])
axes[0][1].plot(splopter.raw_data['V'])
axes[1][1].plot(splopter.raw_data['I'])

In [9]:
# Test bed for stripping/pruning functions

backup_of_interest = list(biases.keys())[2]
current = currents[backup_of_interest]
voltage = biases[backup_of_interest]
prev_current = 4.001464046845688

fig, axes = plt.subplots(2, 1, sharex=True, sharey=True)

axes[0].plot(current, label=f'{bu_folder}')
axes[0].axhline(prev_current, **c.AX_LINE_DEFAULTS)

print(round(voltage[-1], 2))

if round(voltage[-1], 2) >= 10.00:

    big_spikes = np.where(np.abs(current) > 1000)
    if len(big_spikes[0]) == 0:
        pass
    if len(big_spikes[0]) >= 1:
        end_spike = big_spikes[0][np.argmax(np.abs(current[big_spikes]))]
        print(big_spikes, end_spike)
        axes[1].plot(np.arange(end_spike), current[:end_spike], label=f'pre-spike')
        axes[1].plot(np.arange(end_spike+1, len(current)), current[end_spike+1:], label=f'post-spike')

        axes[1].legend()
#     axes[0].set_ylim(-300, 50)

do_current_concat = True
indiced_oi = None
new_current, start_point = strip_head(current, prev_current)

axes[1].plot(np.arange(start_point, len(current)), new_current)
for i in range(2):
    axes[i].axvline(start_point, **c.AX_LINE_DEFAULTS)

0.0


NameError: name 'strip_head' is not defined

In [15]:
def strip_head(current, prev_current):
    print(f'prev_current = {prev_current}')
    
    # Primary method - find first point roughly equal to previous end value
    start_points = np.where(np.logical_and(current >= prev_current * 0.8, current <= prev_current * 1.2))
    if len(start_points[0]) > 0:
        for start_point in start_points[0]:
            if not any(current[start_point:] == 0.0):
                print('Successfully used primary method')
                return current[start_point:], start_point
            else:
                print(f'Failed with start_point {start_point}')
    
    # Secondary method - find last zero in current array and take the succeeding value
    zero_indices = np.where(current == 0.0)[0]
    if current[-1] != 0.0 and len(zero_indices) > 0:
        # Take start point to be the value one place along from the final 0
        start_point = np.max(zero_indices + 1)
        print('Successfully used secondary method')
        return current[start_point:], start_point
    
    return None, None 

def strip_tail(current, threshold=2000):
    # Strip tail section if present
    big_spikes = np.where(np.abs(current) > threshold)
    if len(big_spikes[0]) == 0:
        print('No spikes found on tail.')
    elif len(big_spikes[0]) >= 1:
        end_spike = big_spikes[0][np.argmax(np.abs(current[big_spikes]))]
        current = current[:end_spike]
    return current

def prune_voltage(voltage):
    bias_diff = np.abs(np.diff(voltage))

    # Find spikes in voltage data
    bias_diff_ext = np.append(bias_diff, 0)
    bias_diff_offset = np.append(0, bias_diff)
    spikes = np.where(np.logical_and(bias_diff_ext > 5, bias_diff_offset > 5))

    for spike in spikes:
        if spike == 0: 
            voltage[spike] = voltage[1]
        elif spike == len(voltage):
            voltage[spike] = 10.05
        else:
            voltage[spike] = np.mean([voltage[spike+1], voltage[spike-1]])
    return voltage

In [18]:
concatted_current = np.array([])
concatted_voltage = np.array([])
prev_current = 0.0
prev_voltage = 0.0

for i, backup_of_interest in enumerate(biases.keys()):
    
    current = currents[backup_of_interest].copy()
    voltage = biases[backup_of_interest].copy()
    finished_fl = False
    
    if i == 0:
        # If first step then whole current array is left for concatting and final 
        # current value is stored for next iteration to assist in stripping
        prev_current = current[-1]
    
    fig, ax = plt.subplots()
    ax.plot(np.arange(len(current)), current)
    
    if round(voltage[-1], 2) >= 9.95 or i+1 == len(biases):
        # If final step then strip off the end spike, zeroes and appended IV, if 
        # present
        print(f'Detected finish, stripping tail for backup {i}')
        finished_fl = True
        current = strip_tail(current)
        ax.plot(np.arange(len(current)), current)
    
    if i >= 1:
        # If not the first step then strip the leading zeroes/spikes
        print(f'Stripping head for backup {i}')
        post_tailstrip_length = len(current)
        current, start_point = strip_head(current, prev_current)
        prev_current = current[-1]
    
        print(start_point)
        ax.plot(np.arange(start_point, post_tailstrip_length), current)
    
    concatted_current = np.append(concatted_current, current)
    
    if finished_fl:
        # Final version of voltage is stored in completed run, needs some minor pruning
        concatted_voltage = prune_voltage(voltage)
        break
      

fig, axes = plt.subplots(2, 1)
axes[0].plot(concatted_current)
axes[1].plot(concatted_voltage, color='r')

           
    

Detected finish, stripping tail for backup 1
Stripping head for backup 1
prev_current = 7.856442390517522
Failed with start_point 1
Successfully used primary method
902


In [27]:
print(len(splopter.iv_data['V']), len(splopter.iv_data['I']), len(splopter.iv_data['t']))

spikes = np.where(np.abs(np.diff(splopter.iv_data['V'])) < 5)
print(f'length of spikes {len(spikes[0])}')
iv_data_backup = splopter.iv_data['V']
# splopter.iv_data['V'] = splopter.iv_data['V'][spikes]

plt.plot(splopter.iv_data['t'], splopter.iv_data['V'])
plt.show()


533 533 533
length of spikes 532


In [29]:
splopter.iv_data.plot()

The below is the current method for removing spikes from the concatted bias data. It seems that there's a length mismatch between the bias and current arrays, which is causing some headaches. In a specific test example removing the spikes seems to prevent this, but this doesn't seem to apply generally as a non-restarted (and therefore non-concattable) run is seeing the same issue. Will need to check the homogenisation method to check what it's doing. 

In [57]:
splopt = splopter

# Inital rought method for removing the spikes in voltage data. 
print(len(splopt.iv_data['V']), len(splopt.iv_data['I']), len(splopt.iv_data['t']))
iv_data_backup = copy.deepcopy(splopt.iv_data['V'])

bias_diff = np.abs(np.diff(splopt.iv_data['V']))
print(f'length of spikes {len(bias_diff)}')


# averaged_diff = np.zeros_like(splopt.iv_data['V'])
bias_diff_ext = np.append(np.abs(np.diff(splopt.iv_data['V'])), 0)
bias_diff_offset = np.append(0, np.abs(np.diff(splopt.iv_data['V'])))
spikes = np.where(np.logical_and(bias_diff_ext > 5, bias_diff_offset > 5))

for spike in spikes:
    iv_data_backup[spike] = np.mean([splopt.iv_data['V'][spike+1], splopt.iv_data['V'][spike-1]])
    
print(splopt.iv_data['V'][spikes])

fig, ax = plt.subplots(2, sharex=True)
ax[0].plot(bias_diff_ext)
ax[0].plot(bias_diff_offset)
ax[1].plot(np.logical_and(bias_diff_ext > 5, bias_diff_offset > 5))

fig, ax = plt.subplots(2, sharex=True, sharey=True)
ax[0].plot(splopt.iv_data['V'])
ax[1].plot(iv_data_backup)

# splopter.iv_data['V'] = splopter.iv_data['V'][spikes]

# plt.plot(splopter_single.iv_data['t'], splopter_single.iv_data['V'])
plt.show()

586 585 585
length of spikes 585
[-9.9]


In [68]:
splopter_single.raw_data.plot()

ValueError: shape mismatch: objects cannot be broadcast to a single shape